In [4]:
using Random
using Printf
using Dates

function MSA_to_TSP(sequences)
  node = []
  for i in 1:length(sequences)
    push!(node,sequences[i])
  end

  graph = Array{Float64, 2}(undef, length(node), length(node))
  max_score = 0
  for i in 1:length(node)
    graph[i,i] = 0
    for j in i+1:length(node)
      score, align1, align2 = get_alignment_score(node[i],node[j]) 
      graph[i,j] = score
      graph[j,i] = score
      if max_score < score 
                max_score = score
      end
    end
  end
  for i in 1:length(node)
    for j in i+1:length(node)
      graph[i,j] = max_score - graph[i,j] + 1
      graph[j,i] = max_score - graph[j,i] + 1
    end
  end    
  return graph
end

function get_alignment_score(v, w, match_penalty=1, mismatch_penalty=-1, deletion_penalty=-1)
    n1 = length(v)
    n2 = length(w)
    #if !use_preallocated_matrices
    s = zeros(Float64, n1+1, n2+1)
    b = zeros(Float64, n1+1, n2+1)
    #end

    for i in 1:(n1+1)
        s[i,1] = (i-1) * deletion_penalty
        b[i,1] = 2
    end
    for j in 1:(n2+1)
        s[1,j] = (j-1) * deletion_penalty
        b[1,j] = 3
    end

    for i in 2:(n1+1)
        for j in 2:(n2+1)
            if v[i-1] == w[j-1]
                ms = s[i-1,j-1] + match_penalty
            else
                # ignore cases where a letter is paired with a gap
                # do not consider this a mismatch
                # if v[i-1] != '-' && w[j-1] != '-'
                #     ms = s[i-1,j-1] + mismatch_penalty
                # else
                #     # if a letter is paired with a gap, add no penalty
                #     ms = s[i-1,j-1] #+ match_penalty # + 0.5 * mismatch_penalty
                # end
                ms = s[i-1,j-1] + mismatch_penalty
            end
            test = [ms, s[i-1,j] + deletion_penalty, s[i,j-1] + deletion_penalty]
            p = argmax(test)
            s[i,j] = test[p]
            b[i,j] = p
        end
    end

    i = n1+1
    j = n2+1
    sv = []
    sw = []
    while(i > 1 || j > 1)
        p = b[i,j]
        if (p == 1)
            i = i-1
            j = j-1
            push!(sv, v[i])
            push!(sw, w[j])
        elseif p == 2
            i=i-1
            push!(sv, v[i])
            push!(sw, "-")
        elseif p == 3
            j = j-1
            push!(sv, "-")
            push!(sw, w[j])
        else
            break
        end
    end

    return (s[n1+1,n2+1], join(reverse(sv)), join(reverse(sw)))
end

# Return the ant colony with cities initialized to each ant
function create_colony(num_ants, num_nodes)
    colony = []
    for i in 1 : num_ants
       push!(colony, Dict("path"=>[rand(1:num_nodes)], "distance" => 0))
    end
    return colony
end

function create_pheror_matrix(num_nodes)
    pheromone = zeros(Float64, num_nodes, num_nodes)
    for i in 1: num_nodes
        for j in 1: num_nodes
            pheromone[i,j] = 1/num_nodes
        end
    end
    return pheromone
end

function calculate_proba(num_nodes, pheromone, distance_matrix, alpha, beta)
    probability = zeros(Float64, num_nodes, num_nodes)
    for i in 1: num_nodes
        for j in 1: num_nodes
            probability[i,j] = (pheromone[i,j]^alpha) * (distance_matrix[i,j]^-beta)
            probability[j,i] = probability[i,j]
        end
    end
    return probability
end

function calculate_proba_ant(pheromone, distance_matrix, unvisited_nodes, current_node, proba, alpha, beta)
  sigma = 0.0
  for unvisited_node in unvisited_nodes
    sigma += (pheromone[current_node,unvisited_node]^alpha) * (distance_matrix[current_node,unvisited_node]^-beta)
  end
  proba_ant = proba[current_node,:]/sigma
  return proba_ant
end

function find_best_path(n_ants, colony)
  bpath = []
  best_distance = Inf32
  typeof(best_distance)
  idx_best = 0
  for i=1: n_ants
    if colony[i]["distance"] < best_distance
      best_distance = colony[i]["distance"]
      bpath = colony[i]["path"]
      idx_best = i
    end
  end
  best_path = Dict("path"=> bpath, "distance"=> best_distance, "ant"=> idx_best)
  return best_path
end

function update_pheror_matrix(num_nodes, n_ants, pheromone, distance_matrix, colony, Q, decay)
  depositpher = 0.0
  for i=1: n_ants
    ant = i
    for j= 1:(length(colony[ant]["path"])-1)
      src = colony[ant]["path"][j]
      dest = colony[ant]["path"][j+1]
      pheromone[src,dest] += Q/colony[i]["distance"]
    end
    depositpher += Q/colony[i]["distance"]
    for i= 1:num_nodes
      for j= 1:num_nodes
        pheromone[i,j] = (1-decay)*pheromone[i,j]*depositpher
        pheromone[j,i] = pheromone[i,j]
      end
    end
  end
  return pheromone
end

function calculateDist_ant(ant, colony, distmatrix)
  dist = 0
  path = colony[ant]["path"]
  for i= 1:length(path)-1
    dist += distmatrix[path[i],path[i+1]]
  end
  return dist
end

function traverse(ant, num_nodes, colony, pheromone, distance_matrix, proba, alpha, beta)
    unvisited = collect(1:num_nodes)
    current = colony[ant]["path"][1]
    deleteat!(unvisited, findfirst(isequal(current), unvisited))
    for j in 1: num_nodes-1
        if length(unvisited) > 1
            ant_probability = calculate_proba_ant(pheromone, distance_matrix, unvisited, current, proba, alpha, beta)
            prob = map((x) -> ant_probability[x] , unvisited)
            current = unvisited[findmax(prob)[2]]
            deleteat!(unvisited, findfirst(isequal(current), unvisited))
            push!(colony[ant]["path"], current)       
        else
            push!(colony[ant]["path"], unvisited[1])
        end
    end
    colony[ant]["distance"] = calculateDist_ant(ant, colony, distance_matrix)
end

function run1(num_ants, num_nodes, distance_matrix, iterations, Q, decay, alpha, beta)
    pheromone = create_pheror_matrix(num_nodes)
    gbpath = Dict()
    for i= 1: iterations
        colony = create_colony(num_ants, num_nodes)
        probability = calculate_proba(num_nodes, pheromone, distance_matrix, alpha, beta)
        for ant in 1: num_ants
            traverse(ant, num_nodes, colony, pheromone, distance_matrix, probability, alpha, beta)
        end
        # complete update_pheromone_matrix
        pheromone = update_pheror_matrix(num_nodes, num_ants, pheromone, distance_matrix, colony, Q, decay)
        #complete find best path fucntion
        best_path = find_best_path(num_ants, colony)
        
        bpath = best_path
        if i == 1
            gbpath = bpath
        else
            if bpath["distance"] < gbpath["distance"]
                gbpath = bpath
            end
        end
    end 
    # return the best path
    return gbpath["path"]
end

function find_gap_indices(A, alignedA)
    i = 1
    j = 1
    pointer = []
    while j <= length(alignedA)
        if alignedA[j] == '-' && (i > length(A) || A[i] != '-')
            push!(pointer, j)
            j += 1
        else
            j += 1
            i += 1
        end
    end
    return pointer
end

function insert_gaps(S,gap_indices_for_A)
    copy_of_S = S
    if length(gap_indices_for_A) > 0 && length(gap_indices_for_A) > 0
        gap_indices_for_A = sort(gap_indices_for_A)
        for i in gap_indices_for_A
            copy_of_S = string(string(copy_of_S[1:i-1],'-'),copy_of_S[i:end])
        end
    end
    return copy_of_S
end

# this function takes two params: sequences is the original array of input sequences, order is the array output of tsp algorithm of the order of sequences
# notice that the index in both the params are relative to each other
function align_output_sequences(sequences, order) 
    ordered_sequences = Array{String,1}(undef,0)
    for i=1:length(order)
        push!(ordered_sequences,sequences[order[i]])
    end
    aligned_sequences = Array{String,1}(undef,0)
    for i=1:length(ordered_sequences)
        push!(aligned_sequences,ordered_sequences[i])
    end
    for i=1:length(aligned_sequences)-1
        A = aligned_sequences[i]
        B = aligned_sequences[i+1]
        score, alignedA, alignedB = get_alignment_score(A,B)
        gap_indices_for_A = find_gap_indices(A, alignedA)
        # go to all predecessors of A and insert the gaps at same place
        for j = 1:i-1
            S = aligned_sequences[j]
            newly_alinged_S = insert_gaps(S,gap_indices_for_A)
            aligned_sequences[j] = newly_alinged_S
        end
        aligned_sequences[i] = alignedA
        aligned_sequences[i+1] = alignedB
    end  
    return aligned_sequences
end

function Make_Profile(k, t, sequences)
    idxA = zeros(Int64, k)
    idxC = zeros(Int64, k)
    idxG = zeros(Int64, k)
    idxT = zeros(Int64, k)
    idxGap = zeros(Int64, k)

    for i=1:k
        for seq in sequences
            if seq[i] == 'A'
                idxA[i] += 1
            elseif seq[i] == 'C'
                idxC[i] += 1
            elseif seq[i] == 'G'
                idxG[i] += 1
            elseif seq[i] == 'T'
                idxT[i] += 1
            elseif seq[i] == '-'
                idxGap[i] += 1
            end
        end
    end

    return Dict(
        "A" => idxA,
        "C" => idxC,
        "G" => idxG,
        "T" => idxT,
        "-" => idxGap
    )
end

# Score the sequences in A by obtaining the consensus score using Make_Profile
function score_sequences(A::Array{String,1})::Int
    k = length(A[1])    # how long are the sequences
    t = length(A)       # how many sequences
    profile = Make_Profile(k, t, A)
    # score is the sum of the maximum occurring letter (not including gaps) in each position
    score = 0
    for i in 1:k
        a = profile["A"][i]
        t = profile["T"][i]
        g = profile["G"][i]
        c = profile["C"][i]
        gap = profile["-"][i]
#         if max(a, t, g, c) == 4
#             println("inx = ",i)
#             score += 1
#         end
        score += max(a, t, g, c)
#         score += 2(max(a, t, g, c)) - (a + t + c + g) - 2(gap)
    end
    return score
end

function calc_sum_pair(sequences) 
    t = length(sequences)
    k = length(sequences[1])
    score = 0
    for i=1:t
        A = sequences[i]
        for j=i+1:t
            B = sequences[j]
            for idx = 1:k
                if A[idx] == B[idx] && A[idx] != '-'
                    score += 1
                end
            end
        end
    end
    return score
end

function get_sequences_from_file(file_name)
    sequences = []
    flag_first_arrow = true
    open(file_name) do f
        sequence = ""
        for line in eachline(f)
            if startswith(line, '>') 
                if flag_first_arrow
                    flag_first_arrow = false
                    continue
                end
                push!(sequences, sequence)
                sequence = ""
            else
                sequence *= line
            end
        end
        if length(sequence) > 0
            push!(sequences, sequence)
        end
    end
    return sequences
end

function write_to_file(output_filename, aligned_sequences)
    touch(output_filename)
    f = open(output_filename, "w") 
    for i=1:length(aligned_sequences)
        seq_num = string(">s",i)
        write(f,string(seq_num,"\n"))
        write(f,aligned_sequences[i])
        write(f,"\n")
    end
    close(f) 
end

# this is the driver function which take the fasta file as input and writes the output to the output.txt file.
# the default input to this function is input.fasta which should be present in the same folder as this code.
function driver(filename="input1.txt",output_filename = "output1.txt")
    # get the sequences from input file
    input_sequences = get_sequences_from_file(filename)
    # convert the get input_sequences to TSP problem and get the distance_matrix for TSP
    input_distance_matrix = MSA_to_TSP(input_sequences)
    # configuration for ACO
    num_ants = 50
    num_sequences = length(input_sequences)
    iterations = 20
    Q = 0.6
    decay = 0.6
    alpha = 1
    beta = 1
    # run the ant colony optimization, get the order of node traversal
    order_of_alignment = run1(num_ants,num_sequences,input_distance_matrix,iterations,Q,decay,alpha,beta)
    # align output sequences in the order identified by above algorithm
    aligned_sequences = align_output_sequences(input_sequences,order_of_alignment)
    # get the score of the aligned_sequences
    final_score = calc_sum_pair(aligned_sequences)
    # save the output in fasta format in output1.txt
    write_to_file(output_filename, aligned_sequences)
    # print the aligned sequences
    println("Score of aligned sequeces = ",final_score)
    println("The aligned sequences are saved in output file. Printing them here as well:")
    for i=1:length(aligned_sequences)
        println(aligned_sequences[i])
    end  
#     return final_score, aligned_sequences
end
driver()

Score of aligned sequeces = 255
The aligned sequences are saved in output file. Printing them here as well:
-TTGGGCG-CATTGAC---CG-TCC--TTCCTAGCG-T-AT-CATCAA---ACTTGTG-ATTC--TC--TATCTAGAGCAAAATGCGGTGTC-CG-C--TATA-TGGAGATCTATTTCAAAATA
AAT-GG-T-CATAG-C-G-AGATGA--AGCC-A-CG-TGATGGAT-AAT--A-TTGTGCAAACGACC-TTAT-T--AGC--TAT----TGAC-CGTC--GATG-TCCA-A-CGA---GACAATT
-----GTTACATA-TCAG-A-ATCATTAG--AAACGCT-CT---T-AATGGGGTTAAGCAGA-GA-C-TTA-GT-AAG---GAT----TAACTC-CCAAGATGAT--TGACCG---TG-C--TC
-----G----A-A-TCTGTA-TTC-TT----CAA-GCTTC-AACTCCAT-GCACT-A-C-GA--A-CGGTA-GT---G---G------T---T---C---A-CAT--TGACCG---TG------
